In [1]:
## 导入需要的库
import numpy as np
import pandas as pd
import math
import os
import random
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

from statsmodels.stats.weightstats import ztest

In [6]:
lb1 = [0, 6, 9, 13, 19, 25, 26]
lb2 = [4, 21]
lb3 = [1, 2, 3, 5, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 20, 22, 23, 24, 27, 28, 29]

In [3]:
X = pd.read_csv("中国预处理后的数据归一化数据.csv")
X = X.drop(['Unnamed: 0'],axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,226,227,228,229,230,231,232,233,234,235
0,0.639760,0.306344,0.643055,0.404049,0.330518,0.308084,0.310685,0.234917,0.434767,0.496236,...,0.262493,0.400476,0.388387,0.303567,0.432972,0.293022,0.275222,0.409304,0.487033,0.349695
1,0.526038,0.518005,0.634171,0.319361,0.287591,0.565978,0.315569,0.407085,0.439491,0.752988,...,0.263985,0.516160,0.542824,0.439881,0.498713,0.428343,0.388262,0.507750,0.636671,0.244335
2,0.577485,0.445367,0.601349,0.548781,0.432782,0.466347,0.308527,0.437388,0.499600,0.503784,...,0.533018,0.460782,0.533105,0.462667,0.530569,0.417889,0.433309,0.474532,0.592464,0.541625
3,0.372788,0.425158,0.497319,0.618740,0.591934,0.623291,0.538243,0.550853,0.708638,0.712151,...,0.863991,0.528371,0.693132,0.735055,0.621392,0.472567,0.489961,0.795881,0.587294,0.641752
4,0.290665,0.382834,0.459442,0.390895,0.573781,0.605398,0.403858,0.674202,0.359219,0.366133,...,0.450351,0.363986,0.471258,0.675333,0.416333,0.558379,0.281196,0.458233,0.333770,0.442782


In [4]:
## K-means聚类结果
from sklearn.cluster import KMeans
estimator = KMeans(n_clusters = 3)#构造聚类器
estimator.fit(X)#聚类
label2 = estimator.labels_ #获取聚类标签
label2

array([1, 1, 1, 2, 0, 1, 2, 2, 0, 0, 0, 1, 1, 2, 1, 0, 0, 2, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1])

In [5]:
a1, a2, a3 = [], [], []
for j in range(0,len(X)):
    if label2[j] == 2:
        a1.append(j)
    elif label2[j] == 1:
        a2.append(j)
    else:
        a3.append(j)
print(a1)
print(a2)
print(a3)

[3, 6, 7, 13, 17]
[0, 1, 2, 5, 11, 12, 14, 18, 19, 20, 22, 23, 24, 27, 28, 29]
[4, 8, 9, 10, 15, 16, 21, 25, 26]


In [7]:
## 三支聚类
data = X
### 设定正理想解和负理想解
n, m = len(data), len(data.T)

### 计算决策对象到正负理想解的距离
D1 = np.zeros((1,n));D0 = np.zeros((1,n)); Pr = []
for i in range(0,n):
    D1[0][i] = np.sqrt(np.sum(np.square(1 - data.iloc[i]))) 
    D0[0][i] = np.sqrt(np.sum(np.square(data.iloc[i])))
    Pr.append(D0[0][i]/(D0[0][i] + D1[0][i]))
print(Pr)

[0.38731884560995267, 0.48515364427723945, 0.5177676316460668, 0.5934710852331934, 0.4678602351780109, 0.5342951273073983, 0.5793072998842645, 0.5138514170853524, 0.44446343987025994, 0.45822917879063235, 0.5040828720469762, 0.4989382703698078, 0.49160273649476155, 0.6346894689927974, 0.5443109748431483, 0.5102572963443291, 0.3800266939330637, 0.5874408135202254, 0.4932478479657997, 0.5229613113461025, 0.41917900291797416, 0.4887983442878263, 0.5161722243902209, 0.5155373972158227, 0.49521055612685755, 0.5022866775902478, 0.505315783330787, 0.4965927950042871, 0.5295964889180934, 0.39354225981232366]


In [8]:
def sunshi(x,singam):
    lamda = np.zeros((len(x),6));
    juhe = np.zeros((1,6));
    
    # 计算相对损失函数
    for i in range(0,len(x)):
        lamda[i,1] = singam[i] * x[i];
        lamda[i,2] = x[i];
        lamda[i,3] = 1-x[i];
        lamda[i,4] = singam[i] * (1-x[i]);
    
    # 计算聚合相对效用函数(均值聚合)
    juhe = np.mean(lamda,axis=0)
    
    return juhe

In [9]:
# 计算不同对象的聚合相对损失函数矩阵
singam = [0.45]*m
lamda = np.zeros((n,6));
for i in range(0,n):
    x = data.iloc[i]
    lamda[i] = sunshi(x ,singam)

#根据聚合相对损失函数计算不同对象的初始阈值
yuzhi = np.zeros((n,2))
for i in range(0,n):
    for j in range(0,len(lamda.T)):
        yuzhi[i][0] = (lamda[i][3]-lamda[i][4])/(lamda[i][3]-lamda[i][4]+lamda[i][1])
        yuzhi[i][1] = (lamda[i][4])/(lamda[i][4]+lamda[i][2]-lamda[i][1])
yuzhi = pd.DataFrame(yuzhi)
yuzhi.head()

,0,1
0,0.664986,0.570588
1,0.566136,0.466242
2,0.531387,0.431526
3,0.449797,0.353696
4,0.585555,0.486073


In [10]:
B1, B2, B3 = [], [], []
for i in range(n):
    if Pr[i] > yuzhi.loc[i][0]:
        B1.append(i)
    elif Pr[i]> yuzhi.loc[i][1] and Pr[i] < yuzhi.loc[i][0]:
        B2.append(i)
    else:
        B3.append(i)
print(B1)
print(B2)
print(B3)

[3, 5, 6, 13, 14, 17, 28]
[1, 2, 7, 10, 11, 12, 15, 18, 19, 21, 22, 23, 24, 25, 26, 27]
[0, 4, 8, 9, 16, 20, 29]


In [11]:
label3 = []
for i in range(0,30):
    if i in B1:
        label3.append(0)
    elif i in B2:
        label3.append(1)
    else:
        label3.append(2)
print(label3)

[2, 1, 1, 0, 2, 0, 0, 1, 2, 2, 1, 1, 1, 0, 0, 1, 2, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 2]


In [12]:
## 基于遗憾理论的灰度关联聚类方法
r1 = np.zeros((n,m))
r2 = np.zeros((n,m))

for i in range(n):
    for j in range(m):
        r1[i][j] = 1/(1.5-data.loc[i][j])
        r2[i][j] = 1/(data.loc[i][j]+0.5)

In [13]:
# 建立感知效用矩阵
def Juzhen(alpha, sigma):
    U = np.zeros((n,m))
    R = r1/(r1+r2)
    G = 1 - np.exp(sigma * abs(r1-1))
    Q = 1 - np.exp(-sigma * abs(r1-1))
    for i in range(n):
        for j in range(m):
            U[i][j] = (R[i][j]) ** alpha + G[i][j] + Q[i][j] 
    return U

In [14]:
U = Juzhen(0.5,0.15)
U

array([[0.75431008, 0.63436585, 0.75536746, ..., 0.67412329, 0.70250359,
        0.65141914],
       [0.71623735, 0.71343648, 0.75251055, ..., 0.70984018, 0.75331636,
        0.60912237],
       [0.73383199, 0.68745955, 0.74178825, ..., 0.69803035, 0.73884144,
        0.72163083],
       ...,
       [0.68748728, 0.66365852, 0.75419257, ..., 0.73277984, 0.7404126 ,
        0.72935142],
       [0.68243158, 0.69765829, 0.74387152, ..., 0.70131022, 0.6987265 ,
        0.72779294],
       [0.6902746 , 0.63784927, 0.71478698, ..., 0.65423609, 0.65414339,
        0.66201984]])

In [15]:
# 计算聚类系数
u = []
w = [0.1,0.2,0.5,0.05,0.15]
for i in range(n):
    u.append(np.sum(w * U[i][m-5:]))
print(u)

[0.6571678570136624, 0.6853151978726209, 0.6985410177755942, 0.7610265654738317, 0.6788979680169183, 0.7044466653855682, 0.7365167896658834, 0.7139082340707376, 0.6817599093948655, 0.6712785291827652, 0.6760780324752478, 0.7093535293433841, 0.6991525546207205, 0.7618293036488042, 0.7120739004506849, 0.7055211524401681, 0.6203721443042579, 0.7385976687166376, 0.6882960731423817, 0.7072152560052077, 0.6869921911390094, 0.6769156549332996, 0.7029487689849325, 0.6998585967809078, 0.68796476111613, 0.6984071988354588, 0.6981472265287815, 0.712647112319842, 0.7022616961492026, 0.6449681320250483]


In [16]:
C1, C2 = [], []
for i in range(n):
    if u[i] > np.mean(u):
        C1.append(i)
    else:
        C2.append(i)
print(C1)
print(C2)

[2, 3, 5, 6, 7, 11, 12, 13, 14, 15, 17, 19, 22, 23, 25, 26, 27, 28]
[0, 1, 4, 8, 9, 10, 16, 18, 20, 21, 24, 29]


In [17]:
from numpy import unique
from sklearn.cluster import AffinityPropagation
import numpy as np
from sklearn.preprocessing import StandardScaler

model = AffinityPropagation(damping=0.5, max_iter=200, convergence_iter=15, copy=True, preference=None,    
                            affinity='euclidean', verbose=False) # 设置damping : 阻尼系数，取值[0.5,1)
# 匹配模型
model.fit(X)
yhat = model.predict(X)  # yhat为集群结果
clusters = len(unique(yhat))  # 类别

print(yhat)

[0 2 2 1 3 2 1 1 5 2 3 2 2 1 3 3 4 1 2 2 0 3 1 2 2 5 5 2 6 0]


In [22]:
D1, D2, D3, D4, D5, D6, D7 = [], [], [], [], [], [], []
for i in range(n):
    if yhat[i] == 0:
        D1.append(i)
    if yhat[i] == 1:
        D2.append(i)
    if yhat[i] == 2:
        D3.append(i)
    if yhat[i] == 3:
        D4.append(i)
    if yhat[i] == 4:
        D5.append(i)
    if yhat[i] == 5:
        D6.append(i)
    if yhat[i] == 6:
        D7.append(i)
print(D1)
print(D2)
print(D3)
print(D4)
print(D5)
print(D6)
print(D7)

[0, 20, 29]
[3, 6, 7, 13, 17, 22]
[1, 2, 5, 9, 11, 12, 18, 19, 23, 24, 27]
[4, 10, 14, 15, 21]
[16]
[8, 25, 26]
[28]


In [23]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [25]:
# 计算Silhouette系数
silhouette_avg = silhouette_score(X, label3)
print(f"Silhouette Coefficient: {silhouette_avg}")

# 计算Davies-Bouldin指数
dbi = davies_bouldin_score(X, label3)
print(f"Davies-Bouldin Index: {dbi}")

Silhouette Coefficient: 0.06548120317060101
Davies-Bouldin Index: 2.510177216660093
